In [32]:
import findspark
findspark.init()
import pyspark
from pyspark.mllib.fpm import FPGrowth
import re
from random import random
from pyspark import SparkContext
from itertools import islice

In [33]:
# config = pyspark.SparkConf().set('spark.executor.memory', '512M').set('spark.cores.max', '2')\
#             .set('spark.port.maxRetries','200') 
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', port)    
print port,     os.getcwd().split("/")[2]
sc = pyspark.SparkContext(appName="", master='',conf = config )

4150 ajp218


In [3]:
crimeData = sc.textFile('data/crime_large.txt')

print crimeData.take(10)

crimeData = crimeData.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it )

[u'ID,Date,Primary Type,Location Description,Arrest,Domestic,District,Community Area,FBICode', u'5172504,evening,DECEPTIVEPRACTICE,STREET,Arrest_true,Dome_false,019x,6,11', u'5172505,night,CRIMINALDAMAGE,APARTMENT,Arrest_false,Dome_false,020x,3,14', u'5172508,afternoon,ROBBERY,STREET,Arrest_false,Dome_false,011x,26,03', u'5172509,morning,ASSAULT,PARKINGLOTGARAGENONRESID,Arrest_true,Dome_false,003x,69,04A', u'5172510,afternoon,THEFT,GROCERYFOODSTORE,Arrest_true,Dome_false,002x,41,06', u'5172511,evening,BATTERY,APARTMENT,Arrest_true,Dome_false,019x,6,08B', u'5172512,night,DECEPTIVEPRACTICE,STREET,Arrest_false,Dome_false,020x,77,11', u'5172513,evening,CRIMINALDAMAGE,WAREHOUSE,Arrest_false,Dome_false,011x,23,14', u'5172514,morning,CRIMINALDAMAGE,CTAGARAGEOTHERPROPERTY,Arrest_false,Dome_false,024x,1,14']


In [4]:
def splitData(line):
    out = []
    d = line.split(',')
    key = d[0]
    value = d[1:]
    return (key,value)

def keepTheValue(kv):
    out = []
    for v in kv[1]:
        v = ','.join(v)
        out.append(v)
    return out

crimeDB = crimeData.map(splitData)
crimeDB_values = crimeDB.groupByKey().map(keepTheValue)
crimeDB_primaryType = crimeDB_values.map(lambda x: (x[0].split(',')[1], 1)) \
              .reduceByKey(lambda x,y: x + y)

print crimeDB_primaryType.sortBy(lambda x: x[1], ascending=False).take(10)
print '\n'
print 'There are %i different primary types in the dataset' % \
        (crimeDB_primaryType.count())

[(u'NARCOTICS', 22463), (u'THEFT', 18686), (u'BATTERY', 17446), (u'CRIMINALDAMAGE', 11652), (u'OTHEROFFENSE', 6810), (u'ASSAULT', 5546), (u'BURGLARY', 5540), (u'MOTORVEHICLETHEFT', 4659), (u'CRIMINALTRESPASS', 3577), (u'ROBBERY', 3350)]


There are 28 different primary types in the dataset


In [5]:
crimeDB_val_split = crimeDB_values.map(lambda x: (x[0].split(','))) \
                                  .map(lambda x: list(set(x)))
print crimeDB.take(10)
print '\n'
print crimeDB_values.take(10)
print '\n'
print crimeDB_val_split.take(10)

[(u'5172504', [u'evening', u'DECEPTIVEPRACTICE', u'STREET', u'Arrest_true', u'Dome_false', u'019x', u'6', u'11']), (u'5172505', [u'night', u'CRIMINALDAMAGE', u'APARTMENT', u'Arrest_false', u'Dome_false', u'020x', u'3', u'14']), (u'5172508', [u'afternoon', u'ROBBERY', u'STREET', u'Arrest_false', u'Dome_false', u'011x', u'26', u'03']), (u'5172509', [u'morning', u'ASSAULT', u'PARKINGLOTGARAGENONRESID', u'Arrest_true', u'Dome_false', u'003x', u'69', u'04A']), (u'5172510', [u'afternoon', u'THEFT', u'GROCERYFOODSTORE', u'Arrest_true', u'Dome_false', u'002x', u'41', u'06']), (u'5172511', [u'evening', u'BATTERY', u'APARTMENT', u'Arrest_true', u'Dome_false', u'019x', u'6', u'08B']), (u'5172512', [u'night', u'DECEPTIVEPRACTICE', u'STREET', u'Arrest_false', u'Dome_false', u'020x', u'77', u'11']), (u'5172513', [u'evening', u'CRIMINALDAMAGE', u'WAREHOUSE', u'Arrest_false', u'Dome_false', u'011x', u'23', u'14']), (u'5172514', [u'morning', u'CRIMINALDAMAGE', u'CTAGARAGEOTHERPROPERTY', u'Arrest_false'

In [6]:
no_crimes = crimeDB.count()

crimeDB_Arrest_true = crimeDB.filter(lambda x: x[1][3] == 'Arrest_true')
print crimeDB_Arrest_true.take(10)

no_arrests = crimeDB_Arrest_true.count()

print '\n'
print ('The percentage of arrested crimes is {:3.1f}%'.format( float(no_arrests) / float(no_crimes) * 100 ) )

[(u'5172504', [u'evening', u'DECEPTIVEPRACTICE', u'STREET', u'Arrest_true', u'Dome_false', u'019x', u'6', u'11']), (u'5172509', [u'morning', u'ASSAULT', u'PARKINGLOTGARAGENONRESID', u'Arrest_true', u'Dome_false', u'003x', u'69', u'04A']), (u'5172510', [u'afternoon', u'THEFT', u'GROCERYFOODSTORE', u'Arrest_true', u'Dome_false', u'002x', u'41', u'06']), (u'5172511', [u'evening', u'BATTERY', u'APARTMENT', u'Arrest_true', u'Dome_false', u'019x', u'6', u'08B']), (u'5172518', [u'night', u'BATTERY', u'SIDEWALK', u'Arrest_true', u'Dome_false', u'016x', u'10', u'08B']), (u'5172525', [u'evening', u'CRIMINALDAMAGE', u'OTHER', u'Arrest_true', u'Dome_false', u'009x', u'59', u'14']), (u'5172536', [u'evening', u'THEFT', u'AIRPORTAIRCRAFT', u'Arrest_true', u'Dome_false', u'016x', u'76', u'06']), (u'5172538', [u'evening', u'CRIMINALDAMAGE', u'OTHER', u'Arrest_true', u'Dome_false', u'009x', u'59', u'14']), (u'5172539', [u'afternoon', u'ASSAULT', u'SMALLRETAILSTORE', u'Arrest_true', u'Dome_false', u'003x

In [8]:
model = FPGrowth.train(crimeDB_val_split, minSupport=0.05, numPartitions=10)

result = model.freqItemsets().sortBy(lambda x: x[1], ascending=False).collect()
print('---------------------------')
for fi in result:
    print(fi)


---------------------------
FreqItemset(items=[u'Dome_false'], freq=92860)
FreqItemset(items=[u'Arrest_false'], freq=64595)
FreqItemset(items=[u'Arrest_false', u'Dome_false'], freq=55250)
FreqItemset(items=[u'Arrest_true'], freq=40198)
FreqItemset(items=[u'Arrest_true', u'Dome_false'], freq=37567)
FreqItemset(items=[u'evening'], freq=35660)
FreqItemset(items=[u'afternoon'], freq=33735)
FreqItemset(items=[u'evening', u'Dome_false'], freq=31377)
FreqItemset(items=[u'STREET'], freq=28811)
FreqItemset(items=[u'afternoon', u'Dome_false'], freq=28550)
FreqItemset(items=[u'STREET', u'Dome_false'], freq=27540)
FreqItemset(items=[u'morning'], freq=22598)
FreqItemset(items=[u'NARCOTICS'], freq=22463)
FreqItemset(items=[u'NARCOTICS', u'Dome_false'], freq=22008)
FreqItemset(items=[u'NARCOTICS', u'Arrest_true'], freq=21720)
FreqItemset(items=[u'NARCOTICS', u'Arrest_true', u'Dome_false'], freq=21714)
FreqItemset(items=[u'18'], freq=20685)
FreqItemset(items=[u'18', u'Dome_false'], freq=20533)
FreqIte

In [34]:
print '{:3.1f}% of sidewalk arrests are related to narcotics.'.format(7867./9639*100)
print '{:3.1f}% of events at night had no arrest.'.format(11678./17332*100)
print '{:3.1f}% of arrests happen on the street, and only {:3.2f}% of these are related to a domestic case.'.format(12049./40198*100, float(12049-11817)/float(12049)*float(100) )
print '{:3.1f}% of events of type battery had no arrest and were related to a domestic case.'.format(5596./14071*100)
print '{:3.1f}% of thefts occur in the afternoon.'.format(6724./18686*100)

81.6% of sidewalk arrests are related to narcotics.
67.4% of events at night had no arrest.
30.0% of arrests happen on the street, and only 1.93% of these are related to a domestic case.
39.8% of events of type battery had no arrest and were related to a domestic case.
36.0% of thefts occur in the afternoon.


In [35]:
sc.stop()